Load the csv into dataframe

In [1]:
import pandas as pd 

file_path = '..\data\StudentPerformanceFactors.csv'

df = pd.read_csv(file_path)

In [3]:
import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')

# Write the pandas DataFrame to the SQLite database as a table
df.to_sql('student_performance', conn, index=False, if_exists='replace')

6607

In [4]:
query = "SELECT * FROM student_performance"

# Execute SQL query
result_df = pd.read_sql(query, conn)

# Display results
display(result_df)

,Hours_Studied,Attendance,Parental_Involvement,Access_to_Resources,Extracurricular_Activities,Sleep_Hours,Previous_Scores,Motivation_Level,Internet_Access,Tutoring_Sessions,Family_Income,Teacher_Quality,School_Type,Peer_Influence,Physical_Activity,Learning_Disabilities,Parental_Education_Level,Distance_from_Home,Gender,Exam_Score
0,23,84,Low,High,No,7,73,Low,Yes,0,Low,Medium,Public,Positive,3,No,High School,Near,Male,67
1,19,64,Low,Medium,No,8,59,Low,Yes,2,Medium,Medium,Public,Negative,4,No,College,Moderate,Female,61
2,24,98,Medium,Medium,Yes,7,91,Medium,Yes,2,Medium,Medium,Public,Neutral,4,No,Postgraduate,Near,Male,74
3,29,89,Low,Medium,Yes,8,98,Medium,Yes,1,Medium,Medium,Public,Negative,4,No,High School,Moderate,Male,71
4,19,92,Medium,Medium,Yes,6,65,Medium,Yes,3,Medium,High,Public,Neutral,4,No,College,Near,Female,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6602,25,69,High,Medium,No,7,76,Medium,Yes,1,High,Medium,Public,Positive,2,No,High School,Near,Female,68
6603,23,76,High,Medium,No,8,81,Medium,Yes,3,Low,High,Public,Positive,2,No,High School,Near,Female,69
6604,20,90,Medium,Low,Yes,6,65,Low,Yes,3,Low,Medium,Public,Negative,2,No,Postgraduate,Near,Female,68
6605,10,86,High,High,Yes,6,91,High,Yes,2,Low,Medium,Private,Positive,3,No,High School,Far,Female,68


Average Exam Score by Study and Extracurricular

In [5]:
## avg_exam_score_by_study_and_extracurricular

avg_exam_score_by_study_and_extracurricular_query = """
SELECT 
	SP.hours_studied
	, AVG(SP.exam_score)	AS avg_exam_score
FROM student_performance SP
WHERE SP.hours_studied > 10
	AND SP.extracurricular_activities = 'Yes'
GROUP BY SP.hours_studied
ORDER BY hours_studied DESC
"""

# Execute SQL query
avg_exam_score_by_study_and_extracurricular_df = pd.read_sql(avg_exam_score_by_study_and_extracurricular_query, conn)

display(avg_exam_score_by_study_and_extracurricular_df)

,Hours_Studied,avg_exam_score
0,43,78.000000
1,39,75.000000
2,38,73.500000
3,37,73.000000
4,36,70.428571
5,35,72.312500
6,34,71.187500
7,33,70.333333
8,32,71.325000
9,31,70.553191


In [6]:
## avg_exam_score_by_hours_studied_range

avg_exam_score_by_hours_studied_range_query = """
SELECT
	CASE
		WHEN SP.hours_studied BETWEEN 1 AND 5 THEN '1-5 hours'
		WHEN SP.hours_studied BETWEEN 6 AND 10 THEN '6-10 hours'
		WHEN SP.hours_studied BETWEEN 11 AND 15 THEN '11-15 hours'
		WHEN SP.hours_studied >= 16 THEN '16+ hours'
	END AS hours_studied_range
	, AVG(SP.exam_score)		AS avg_exam_score
FROM student_performance SP
GROUP BY hours_studied_range
ORDER BY avg_exam_score DESC
"""

# Execute SQL query
avg_exam_score_by_hours_studied_range_df = pd.read_sql(avg_exam_score_by_hours_studied_range_query, conn)

display(avg_exam_score_by_hours_studied_range_df)

,hours_studied_range,avg_exam_score
0,16+ hours,67.923363
1,11-15 hours,65.204386
2,6-10 hours,64.225490
3,1-5 hours,62.627119


In [7]:
## student_exam_ranking

student_exam_ranking_query = """
SELECT
	SP.attendance
	, SP.hours_studied
	, SP.sleep_hours
	, SP.tutoring_sessions
	, DENSE_RANK() OVER (ORDER BY exam_score DESC) AS exam_rank
FROM student_performance SP
LIMIT 30
"""

# Execute SQL query
student_exam_ranking_df = pd.read_sql(student_exam_ranking_query, conn)

display(student_exam_ranking_df)

,Attendance,Hours_Studied,Sleep_Hours,Tutoring_Sessions,exam_rank
0,98,27,6,5,1
1,89,18,4,3,2
2,83,23,4,1,3
3,90,14,8,4,3
4,90,28,9,0,4
5,96,28,4,1,4
6,83,16,8,2,4
7,83,15,7,2,5
8,74,21,6,1,5
9,99,25,7,0,5
